In [19]:
import mysql.connector
from sqlalchemy import create_engine
import sqlalchemy as sa
from pandas import DataFrame
import config

In [25]:
def query_database(sql_query, db_config):
    # Connect to the database
    try:
        # Create SQLAlchemy engine
        engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}", 
                               pool_size=10, 
                               max_overflow=20
                               )
        # # Connect to the database using the engine
        conn = engine.connect()
        resoverall = conn.execute(sql_query)
        df = DataFrame(resoverall.fetchall())

    except mysql.connector.Error as e:
        return None, e
    except Exception as e:
        return None, e
    else:
        return df, None

In [ ]:
Total_Outstanding_Amount_by_Market = "SELECT market, SUM(outstanding_amount) as total_outstanding FROM customers GROUP BY market;"

df, error = query_database(sql_query=Total_Outstanding_Amount_by_Market, 
                           db_config={"host": config.DB_HOST,
                                      "user": config.DB_USER,
                                      "password": config.DB_PASSWORD.replace('@', '%40'),
                                      "database": config.DB_NAME
                                    }
                           )

if df is not None:
   print(df.head(100))
else:
   print(error)

In [28]:
Total_Outstanding_Amount_by_Market = "SELECT market, SUM(outstanding_amount) as total_outstanding FROM Customers GROUP BY market;"
Number_of_Blocked_Customers = "SELECT COUNT(*) as blocked_customers FROM Customers WHERE is_blocked = 1;"
Most_Used_Platform_by_Customers = "SELECT last_used_platform, COUNT(*) as count FROM Customers GROUP BY last_used_platform ORDER BY count DESC LIMIT 1;"
Total_Orders_by_Customer = "SELECT customer_id, COUNT(*) as total_orders FROM Orders GROUP BY customer_id;"
Total_Earnings_by_Merchant = "SELECT merchant_id, SUM(merchant_earning) as total_earning FROM Orders GROUP BY merchant_id;"
Average_Delivery_Time = "SELECT AVG(TIMESTAMPDIFF(MINUTE, order_time, delivery_time)) as avg_delivery_time FROM Orders;"
Total_Tasks_by_Agent = "SELECT agent_id, COUNT(*) as total_tasks FROM Deliveries GROUP BY agent_id;"
Total_Earnings_by_Agent = "SELECT agent_id, SUM(earning) as total_earning FROM Deliveries GROUP BY agent_id;"
Average_Rating_by_Agent = "SELECT agent_id, AVG(rating) as avg_rating FROM Deliveries GROUP BY agent_id;"
Total_Distance_Covered_by_Agent = "SELECT agent_id, SUM(distance) as total_distance FROM Deliveries GROUP BY agent_id;"

queries = [Total_Outstanding_Amount_by_Market, Number_of_Blocked_Customers, Most_Used_Platform_by_Customers, 
           Total_Orders_by_Customer, Total_Earnings_by_Merchant, Average_Delivery_Time, Total_Tasks_by_Agent,
           Total_Earnings_by_Agent, Average_Rating_by_Agent, Total_Distance_Covered_by_Agent
           ]

In [32]:
queries = [
    {
        "query": """
        SELECT c.market, COUNT(*) AS total_customers
        FROM Customers c
        GROUP BY c.market
        ORDER BY total_customers DESC;
        """,
        "title": "Total Customers by Market",
        "description": "This query calculates the total number of customers for each market."
    },
    {
        "query": """
        SELECT o.category_name, SUM(o.total_price) AS total_revenue
        FROM Orders o
        GROUP BY o.category_name
        ORDER BY total_revenue DESC;
        """,
        "title": "Total Revenue by Order Category",
        "description": "This query calculates the total revenue generated from each order category."
    },
    {
        "query": """
        SELECT d.task_type, AVG(d.distance(m)) AS avg_distance
        FROM Deliveries d
        GROUP BY d.task_type;
        """,
        "title": "Average Distance Traveled by Delivery Task Type",
        "description": "This query calculates the average distance traveled for each delivery task type."
    },
    {
        "query": """
        SELECT o.merchant_id, SUM(o.merchant_earning) AS total_earnings
        FROM Orders o
        GROUP BY o.merchant_id
        ORDER BY total_earnings DESC;
        """,
        "title": "Total Earnings by Merchant",
        "description": "This query calculates the total earnings for each merchant."
    },
    {
        "query": """
        SELECT c.loyalty_points, COUNT(*) AS loyal_customers
        FROM Customers c
        WHERE c.loyalty_points > 0
        GROUP BY c.loyalty_points
        ORDER BY loyal_customers DESC;
        """,
        "title": "Number of Loyal Customers by Loyalty Points",
        "description": "This query identifies the number of customers with loyalty points and groups them by their points."
    },
    {
        "query": """
        SELECT o.order_status, COUNT(*) AS order_count
        FROM Orders o
        GROUP BY o.order_status
        ORDER BY order_count DESC;
        """,
        "title": "Order Count by Order Status",
        "description": "This query calculates the number of orders for each order status."
    },
    {
        "query": """
        SELECT d.agent_name, AVG(d.rating) AS avg_rating
        FROM Deliveries d
        WHERE d.rating > 0
        GROUP BY d.agent_name
        ORDER BY avg_rating DESC;
        """,
        "title": "Average Rating by Delivery Agent",
        "description": "This query calculates the average rating received by each delivery agent (excluding agents with no ratings)."
    },
    {
        "query": """
        SELECT o.payment_method, SUM(o.total_price) AS total_sales
        FROM Orders o
        GROUP BY o.payment_method
        ORDER BY total_sales DESC;
        """,
        "title": "Total Sales by Payment Method",
        "description": "This query calculates the total sales amount for each payment method used."
    }
]


In [34]:
for inx, query in enumerate(queries):
    df, error = query_database(sql_query=query['query'], 
                           db_config={"host": config.DB_HOST,
                                      "user": config.DB_USER,
                                      "password": config.DB_PASSWORD.replace('@', '%40'),
                                      "database": config.DB_NAME
                                    }
                           )

    if df is not None:
       print(f"====================================\n{query['title']}\n =====================================")
       print(df.head(100))
    else:
       print(error)
    

Total Customers by Market
   market  total_customers
0       1             4177
1       2             1095
Total Revenue by Order Category
          category_name  total_revenue
0         Flour & Sugar    103333325.0
1     Cooking Fat & Oil     23886468.0
2        Rice & Cereals      5651920.0
3   Noodles & Spaghetti      1783200.0
4      Salt & Seasoning      1389826.0
5           Meat & Fish       773947.0
6   Fruits & Vegetables       562906.0
7               Spreads       561345.0
8             Beverages       557513.0
9             Packaging       316376.0
10   Cleaning & Hygiene       278628.0
11           Vegetables       139600.0
12                 Wine        18200.0
13       Beer & Bitters        13100.0
14                 None         1194.0
(mysql.connector.errors.ProgrammingError) 1305 (42000): FUNCTION d.distance does not exist
[SQL: 
        SELECT d.task_type, AVG(d.distance(m)) AS avg_distance
        FROM Deliveries d
        GROUP BY d.task_type;
        ]
(Backgroun